In [1]:
!pip install --user --upgrade tensorflow-gpu

In [2]:

!pip install tensorflow keras numpy np_utils


  Created wheel for np-utils: filename=np_utils-0.5.12.1-py3-none-any.whl size=57125 sha256=18505b99c5d6b60141e215f09704ba76736c9234bf7aca74c60ca7a967b6eb4e
  Stored in directory: c:\users\romit\appdata\local\pip\cache\wheels\8c\4e\ef\095c24693723c329f4cdc1079861cdbb2487d4b41b2496a4e7
Successfully built np-utils


In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D


In [3]:
from keras.utils import np_utils
import pandas as pd
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K

In [4]:
    data = pd.read_csv("C:\\Users\\romit\\Eckovation ML\\DevanagariHandwrittenCharacterDataset\\data.csv")
    dataset = np.array(data)
    np.random.shuffle(dataset)
    X = dataset
    Y = dataset
    X = X[:, 0:1024]
    Y = Y[:, 1024]

    X_train = X[0:70000, :]
    X_train = X_train / 255.
    X_test = X[70000:72001, :]
    X_test = X_test / 255.

    # Reshape
    Y = Y.reshape(Y.shape[0], 1)
    Y_train = Y[0:70000, :]
    Y_train = Y_train.T
    Y_test = Y[70000:72001, :]
    Y_test = Y_test.T

In [5]:
    print("number of training examples = " + str(X_train.shape[0]))
    print("number of test examples = " + str(X_test.shape[0]))
    print("X_train shape: " + str(X_train.shape))
    print("Y_train shape: " + str(Y_train.shape))
    print("X_test shape: " + str(X_test.shape))
    print("Y_test shape: " + str(Y_test.shape))

number of training examples = 70000
number of test examples = 2000
X_train shape: (70000, 1024)
Y_train shape: (1, 70000)
X_test shape: (2000, 1024)
Y_test shape: (1, 2000)


In [6]:
    image_x = 32
    image_y = 32

    train_y = np_utils.to_categorical(Y_train)
    test_y = np_utils.to_categorical(Y_test)
    train_y = train_y.reshape(train_y.shape[1], train_y.shape[2])
    test_y = test_y.reshape(test_y.shape[1], test_y.shape[2])
    X_train = X_train.reshape(X_train.shape[0], 32, 32, 1)
    X_test = X_test.reshape(X_test.shape[0], 32, 32, 1)

In [7]:
    print("X_train shape: " + str(X_train.shape))
    print("Y_train shape: " + str(train_y.shape))

X_train shape: (70000, 32, 32, 1)
Y_train shape: (70000, 37)


In [8]:
def keras_model(image_x,image_y):
    num_of_classes = 37
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5, 5), input_shape=(image_x, image_y, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same'))
    model.add(Flatten())
    model.add(Dense(num_of_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath = "devanagari.h5"
    checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    #checkpoint2 = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint1]

    return model, callbacks_list

In [9]:
    model, callbacks_list = keras_model(image_x, image_y)
    model.fit(X_train, train_y, validation_data=(X_test, test_y), epochs=8, batch_size=64,callbacks=callbacks_list)
    scores = model.evaluate(X_test, test_y, verbose=0)
    print("CNN Error: %.2f%%" % (100 - scores[1] * 100))
    model.save('devanagari.h5')


Epoch 1/8
1094/1094 [==============================] - 40s 36ms/step - loss: 1.4947 - accuracy: 0.5973 - val_loss: 0.3044 - val_accuracy: 0.9075
Epoch 2/8
1094/1094 [==============================] - 41s 37ms/step - loss: 0.2426 - accuracy: 0.9285 - val_loss: 0.1933 - val_accuracy: 0.9425
Epoch 3/8
1094/1094 [==============================] - 39s 36ms/step - loss: 0.1517 - accuracy: 0.9547 - val_loss: 0.1495 - val_accuracy: 0.9595
Epoch 4/8
1094/1094 [==============================] - 43s 40ms/step - loss: 0.1107 - accuracy: 0.9673 - val_loss: 0.1191 - val_accuracy: 0.9670
Epoch 5/8
1094/1094 [==============================] - 39s 35ms/step - loss: 0.0830 - accuracy: 0.9745 - val_loss: 0.1055 - val_accuracy: 0.9685
Epoch 6/8
1094/1094 [==============================] - 38s 34ms/step - loss: 0.0687 - accuracy: 0.9790 - val_loss: 0.1131 - val_accuracy: 0.9625
Epoch 7/8
1094/1094 [==============================] - 37s 34ms/step - loss: 0.0561 - accuracy: 0.9821 - val_loss: 0.1134 - val_ac